<a href="https://colab.research.google.com/github/venkatasl/AIML_TRAINING_VENKAT/blob/venkat_creation/DRDO2024_FeatureVisualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Visualization

What do the different convolutional filters of a model represent? In this notebook, we find out by visualizing the filters.

In [ ]:
# import everything
import torch
import torchvision.transforms as transforms
import torchvision.transforms.v2 as v2
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import math

gpu = False # turn to True if using a gpu kernel

In [ ]:
# Load a pre-trained model
model = models.vgg16(pretrained=True)

# put the model on the gpu
if gpu:
  model.cuda()

model

VGG16 has 39 layers in total. Let us write a function to propagate an input forward to the desired layer and get the features from there.



In [ ]:
def get_features(model, input, layernum):
  index = 0
  for layer in model.features.children():
    input = layer(input)
    if index==layernum:
      return input
    index += 1

  input = model.avgpool(input)
  if index==layernum:
    return input
  index += 1

  input = torch.flatten(input, 1)

  for layer in model.classifier.children():
    input = layer(input)
    if index==layernum:
      return input
    index += 1
  return index # if layernum is bigger than the number of layers, it will return how many layers are there


Let us define some helper functions:

In [ ]:
# Define the image transformation. This is important because this is how the model was trained
preprocess = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    # The below values are based on the mean and st.deviation of the ImageNet dataset
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# inverse of the preprocess transform
inversetransform = transforms.Compose([
        transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.255]),
        transforms.ToPILImage()])

Let us define some loss functions nd regularizations

In [ ]:
def tv(t,beta=2):
    #gets the smoothness or the edginess of an image
    tv_x = t[:,:,:,1:]-t[:,:,:,:-1]
    tv_y = t[:,:,1:,:]-t[:,:,:-1,:]

    tv_x = tv_x[:,:,:-1,:]
    tv_y = tv_y[:,:,:,:-1]
    if False: print(tv_x)
    tv_2 = tv_x **2 + tv_y **2
    tv = tv_2.pow(beta/2.)
    total = tv.sum()
    return total

def tv2(t):
    #specifically with beta =2
    tv_x = t[:,:,:,1:]-t[:,:,:,:-1]
    tv_y = t[:,:,1:,:]-t[:,:,:-1,:]

    tv_x = tv_x[:,:,:-1,:]
    tv_y = tv_y[:,:,:,:-1]
    tv_x_2 = (tv_x **2).sum()
    tv_y_2 = (tv_y **2).sum()

    if False: print(tv_x)
    tv_2 =  tv_x_2 + tv_y_2
    total = tv_2.sum()
    return total

def alpha_norm(t,alpha=6): #Called p-norm in wikipedia
    a = torch.sum(t.pow(alpha)) # you have to take the alpha^th root also, but it doesn't matter for optimization
    return a

# our goal is the maximize the mean of a particular filter
def total(x_feat, x, alpha_lambda, tv_lambda ):
    return -torch.mean(x_feat) + alpha_lambda*alpha_norm(x) + tv_lambda*tv2(x);



Visualization function:

In [ ]:
def visualize_features(model, layer, filter, alpha_lambda, tv_lambda, lr, iterations=100):
  # let's start with a random image
  x = torch.rand(1,3,224,224) # this is the random image we will optimize
  if gpu:
    x = x.cuda()
  x.requires_grad=True

  # create an optimizer to modify the image:
  optimizer = Adam([x], lr=0.5, weight_decay=1e-6) # create an optimizer for the image

  # now the training iterations!
  for i in range(iterations):
    x_feats = get_features(model, x, layer)[:,filter, :,:] # select the filter we want
    loss = total(x_feats, x, alpha_lambda, tv_lambda)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(f'\r {i} of {iterations} iterations complete', end = '     ')

  return inversetransform(x.squeeze().cpu())

Let us run for a few filters of the 7th layer

In [ ]:
for i in range(128):
  plt.imshow(visualize_features(model, 7 ,i, 1e-6, 1e-4, 0.01, 20))
  plt.show()

## Exercises
1. Try with differnet values for alpha_lambda and tv_lambda and lr
2. Try for different layers. You have to change the values of alpha_lambda, tv_lambda and lr accordingly. See if you can find any recognizable patterns
3. What happens if we change the mean for max in the loss function?